# Demonstração de Super-Resolução com SRFlow
Este notebook demonstra o processo de super-resolução com SRFlow em imagens utilizando o modelo pré-treinado. Cada etapa é explicada para ajudar você a entender o fluxo de trabalho, desde o carregamento das imagens até a avaliação dos resultados.

## 1. Verificar ambiente Python
Primeiro, verificamos qual executável do Python está sendo usado. Isso ajuda a garantir que o ambiente correto está ativo. É esperado que tudo esteja em um ambiente virualizado, na pasta `.venv`.

In [ ]:
import sys
print(sys.executable)

## 2. Importar bibliotecas necessárias e definir funções utilitárias
Importamos todas as bibliotecas necessárias e definimos funções auxiliares para processamento e exibição de imagens.

In [ ]:
import glob
import torch
import numpy as np
from PIL import Image

import Measure
from imresize import imresize
from test import load_model, imread

# Converte uma imagem para o formato RGB
def to_tensor(array):
    return torch.Tensor(np.expand_dims(array.transpose([2, 0, 1]), axis=0).astype(np.float32)) / 255

# Converte um tensor para o formato de imagem
def to_image(t):
    return (np.clip((t[0] if len(t.shape) == 4 else t).detach().cpu().numpy().transpose([1, 2, 0]), 0, 1) * 255).astype(np.uint8)

# Exibe uma imagem em um notebook Jupyter
def imshow(array):
    display(Image.fromarray(array))


## 3. Encontrar Arquivos de Imagem
Buscamos todas as imagens PNG ou JPG pasta raíz. Elas serão usadas como entrada para o processo de super-resolução.

In [ ]:
images = glob.glob("../*.png") +  glob.glob("../*.jpg")

for i, img_path in enumerate(images):
    print(f"Imagem {i}: {img_path}")

## 4. Ler e Exibir uma Imagem de Referência (Ground Truth)
Escolha uma imagem e selecione pelo índice `image_index` para exibimos como a imagem de referência (alta resolução).

In [ ]:
image_index = 0
gt = imread(images[image_index])

imshow(gt)

## 5. Criar e Exibir uma Imagem de Baixa Resolução
Redimensionamos a imagem de referência por um fator de 4 para simular uma entrada de baixa resolução e a exibimos.

In [ ]:
lq = imresize(gt, 1/4)
imshow(lq)

## 6. Carregar o Modelo de Super-Resolução e as Métricas
Carregamos o modelo SRFlow pré-treinado e inicializamos o cálculo das métricas para avaliação. As opções disponíveis são:
- `SRFlow-DA_DF2K_4X.yml`
- `SRFlow-DA_DF2K_8X.yml`

In [ ]:
model, opt = load_model("./confs/SRFlow-DA_DF2K_4X.yml")
measure = Measure.Measure()

## 7. Executar Super-Resolução e Avaliar Resultados
Nesta etapa, usamos uma imagem de alta resolução conhecida (referência) para gerar uma versão de baixa resolução quatro vezes menor e, em seguida, aplicamos o modelo de super-resolução, que gera uma imagem 4 vezes maior que iguala a entrada original. Isso permite comparar a saída do modelo com a imagem original e calcular métricas quantitativas (PSNR, SSIM, LPIPS) para avaliar a qualidade da imagem super-resolvida. Essa avaliação só é possível porque temos acesso à imagem de referência. Então repete-se o processo dez vezes, variando o parâmetro Temperatura de 0.0 à 1.0.

In [ ]:
for temperature in np.linspace(0, 1, num=3):
    result = model.get_sr(lq=to_tensor(lq), heat=temperature)
    sr = to_image(result)
    imshow(sr)
    psnr, ssim, lpips = measure.measure(sr, gt)
    print(f"Temperature: {temperature:0.2f} - PSNR: {psnr:0.1f}, SSIM: {ssim:0.1f}, LPIPS: {lpips:0.2f}\n\n")

## 8. Aumentar a Resolução de uma Única Imagem de Baixa Resolução (Sem Referência)
Em cenários reais, você pode ter apenas uma imagem de baixa resolução e desejar aumentá-la usando o modelo de super-resolução. Nesse caso, é possível gerar uma versão de alta resolução, mas não será possível calcular métricas quantitativas, pois a imagem de referência é desconhecida.

In [ ]:
image_index = 0
lq = imread(images[image_index])

imshow(lq)

result = model.get_sr(lq=to_tensor(lq), heat=0.9)
sr = to_image(result)

imshow(sr)
